In [59]:
# Approach 1: Find derivative of graph, delete points with too high of a change in slope
# Approach 1.1: Chunk graph into pulses, preserve low change portions of a pulse. Modified from Yang's work

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Magic numbers
dy_cutoff = 2000
pulse_size = 20
pulse_dx = .2

# Load file into 2d list
filename = 'P7132033_37'
with open('../data/'+ filename + '.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats
zc_x = list()
zc_y = list()
for x, y in zc_str:
        zc_x.append(float(x) * 1000)
        zc_y.append(float(y))

# Identify pulses
graph = list()
pulse = list()
prev_x = 0
for x, y in zip(zc_x, zc_y):
    if x - prev_x <= pulse_dx:
        pulse.append([x, y])
    elif len(pulse) < pulse_size:
        pulse = [[x, y]]
    else:
        graph.append(pulse)
        pulse = [[x, y]]
    prev_x = x
    
# Get 1st derivative
graph_dy = list()
prev_y = 0
for pulse in graph:
    dy = list()
    for x, y in pulse:
        dy.append(abs(y - prev_y))
        prev_y = y
    graph_dy.append(dy)
    
# Smooth holes
for dy, pulse in zip(graph_dy, graph):
    i = 1
    while i < (len(dy) - 2):
        if dy[i] > dy_cutoff:
            if dy[i - 1] < dy_cutoff:
                if dy[i + 1] < dy_cutoff:
                    pulse[i][1] = (pulse[i - 1][1] + pulse[i + 1][1])/2
                elif dy[i + 2] < dy_cutoff:
                    pulse[i][1] = (pulse[i - 1][1] + pulse[i + 2][1])/2
            elif dy[i - 2] < dy_cutoff:
                if dy[i + 1] < dy_cutoff:
                    pulse[i][1] = (pulse[i - 2][1] + pulse[i + 1][1])/2
                elif dy[i + 2] < dy_cutoff:
                    pulse[i][1] = (pulse[i - 2][1] + pulse[i + 2][1])/2
        i += 1
        
# Clean pulses
clean_graph = list()
for k, pulse in enumerate(graph):
    i = 1
    while i < len(pulse):
        j = i

        # Count neighboring points
        while j < len(pulse) - 1 and graph_dy[k][j] <= dy_cutoff:
            j += 1

        # If there are enough neighbors, it's good
        if j - i >= pulse_size:
            clean_graph.extend(pulse[i:j])

        i = j + 1

# Convert graph to 2 lists of x and y pts
graph_x = list()
graph_y = list()
for pulse in graph:
    for point in pulse:
        graph_x.append(point[0])
        graph_y.append(point[1])
        
# Graph data
trace_noisy = graphs.Scatter(
                            x = graph_x,
                            y = graph_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_noiseless = graphs.Scatter(
                            x = [point[0] for point in clean_graph], 
                            y = [point[1] for point in clean_graph],
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )

trace = [trace_noisy, trace_noiseless]
figure={
    'data': trace
   }
py.iplot(figure, filename=filename)
# return clean_pulses

In [2]:
# Approach 2: Smooth graph, and take comparisons of smoothed and unsmoothed graph. More different points are deleted

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Load file into 2d list
with open('../data/P7132033_37.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
    if float(y) > 44000:
        zc_x.append(float(x))
        zc_y.append(float(y))
    
# Define cutoff dy for ignoring noise
cutoff = 300
    
# Smooth graph- Savitsky-Golay filter
yhat2 = savgol_filter(zc_y, 27, 2)
yhat3 = savgol_filter(zc_y, 17, 3)
yhat4 = savgol_filter(zc_y, 7, 4) # best fit so far

# Compare smoothed and original
i = 0
noiseless_y = list()
noiseless_x = list()
dy = list()
while i < len(zc_x):
    dy.append(abs(zc_y[i] - yhat4[i]))
    if dy[-1] < cutoff:
        noiseless_y.append(zc_y[i])
        noiseless_x.append(zc_x[i])
    i += 1
    
# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = zc_x, 
                            y = yhat4,
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = noiseless_x, 
                            y = noiseless_y,
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )
trace_dy = graphs.Scatter(
                            x = zc_x, 
                            y = dy, 
                            name = 'Derivative- noisy'
                         )
trace_cutoff = graphs.Scatter(
                            x = [zc_x[0], zc_x[-1]], 
                            y = [cutoff, cutoff], 
                            name = 'Cutoff'
                         )
trace = [trace_noisy, trace_smooth, trace_noiseless, trace_cutoff, trace_dy]
py.iplot(trace, filename='P7132033_37_2')

In [49]:
# Approach 3: Smooth graph, and take comparisons of smoothed and unsmoothed graph. Go by chunks and take average difference between smooth and unsmoothed.
# Approach 3.1: Chunk graph into pulses, preserve dense portions of a given pulse

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

# Magic numbers
dy_cutoff = 1000
density = 1000
pulse_size = 19
pulse_dx = .2

# Load file into 2d list
filename = 'P7132033_37'
with open('../data/'+ filename + '.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats
zc_x = list()
zc_y = list()
for x, y in zc_str:
        zc_x.append(float(x) * 1000)
        zc_y.append(float(y))

# Distance functions
def dist(ax, ay, bx, by):
    return numpy.sqrt((ax - bx)**2 + (ay - by)**2)
    
def dista(pair):
    return dist(pair[0][0], pair[0][1], pair[1][0], pair[1][1])

# Segment pulses
pulses = list()
pulse = list()
pulse_lines = list()
prev_x = 0
for x, y in zip(zc_x, zc_y):
    if x - prev_x <= pulse_dx:
        pulse.append([x, y])
    else:
        pulses.append(pulse)
        pulse = [[x, y]]
    prev_x = x
    
# Clean pulses
clean_pulses = list()
smooth_pulses = list()
for pulse in pulses:
    if len(pulse) < pulse_size:
        continue
    
    # Build smooth graph using Savitzky-golay filter
    # Left param is all x values in current pulse, right param is smoothed y values
    # Params are zipped together then converted to list
    # This is the dark side of pythonic code
    smooth_pulse = list(zip([point[0] for point in pulse], savgol_filter([point[1] for point in pulse], 17, 3)))

    # Build clean pulse
    # Iterate through zipped list of smooth_pulse and pulse, producing [[ax, ay],[bx, by]]
    # Keep only those where the absolute distance between pair 1 and pair 2 is less than some density
    # This is the even darker side of pythonic code
    clean_pulse = [pair[0] for pair in zip(pulse, smooth_pulse) if dista(pair) < density]
    
    clean_pulses.extend(clean_pulse)
    smooth_pulses.extend(smooth_pulse)
    
pulses_x = list()
pulses_y = list()
for pulse in pulses:
    for point in pulse:
        pulses_x.append(point[0])
        pulses_y.append(point[1])
        
# Graph data
trace_noisy = graphs.Scatter(
                            x = pulses_x, 
                            y = pulses_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = [point[0] for point in smooth_pulses], 
                            y = [point[1] for point in smooth_pulses],
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = [point[0] for point in clean_pulses], 
                            y = [point[1] for point in clean_pulses],
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )

trace = [trace_noisy, trace_smooth, trace_noiseless]
figure={
    'data': trace
   }
py.iplot(figure, filename=filename)
# return clean_pulses

In [5]:
# Approach 4: Use clustering algorithms. 
# Ineffective so far: K means, DBSCAN

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN

# Load file into 2d list
filename = 'P7132033_37'
with open('../data/'+ filename + '.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
zc = list()
for x, y in zc_str:
        zc_x.append(float(x))
        zc_y.append(float(y))
        zc.append([float(x), float(y)])

# Cluster
# eps- 40 < eps < 1250
clustering = DBSCAN(eps=1250, min_samples=2).fit(zc)

plt.figure(figsize=(20,8))
plt.scatter(zc_x, zc_y, c=clustering.labels_, s=50, cmap='rainbow');

# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )

shapes = list()
layout = graphs.Layout(shapes=shapes)
trace = [trace_noisy]
figure={
    'data': trace,
    'layout': layout
   }
#py.iplot(figure, filename=filename)

In [80]:
# Approach 5: Same as 3, using LOESS instead of savitzky-golay

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from batcall import batcall
from statsmodels.nonparametric.smoothers_lowess import lowess

# Magic numbers
dy_cutoff = 100
cutoff = 2000
avg_d = 3000
pulse_size = 30

# Load file into 2d list
filename = 'P7132033_37'
with open('../data/'+ filename + '.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
for x, y in zc_str:
        zc_x.append(float(x))
        zc_y.append(float(y))
    
# Get dy
prev_y = 0
dy = list()
for y in zc_y:
    dy.append(abs(y-prev_y))
    prev_y = y

# Smooth holes
i = 1
while i < len(dy):
    if dy[i] > dy_cutoff:
        if dy[i - 1] < dy_cutoff:
            if dy[i + 1] < dy_cutoff:
                zc_y[i] = (zc_y[i - 1] + zc_y[i + 1])/2
            elif dy[i + 2] < dy_cutoff:
                zc_y[i] = (zc_y[i - 1] + zc_y[i + 2])/2
        elif dy[i - 2] < dy_cutoff:
            if dy[i + 1] < dy_cutoff:
                zc_y[i] = (zc_y[i - 2] + zc_y[i + 1])/2
            elif dy[i + 2] < dy_cutoff:
                zc_y[i] = (zc_y[i - 2] + zc_y[i + 2])/2
    i += 1

# Smooth graph- Loess
yhat4 = lowess(zc_y, zc_x, 0.05)
yhat4.sort()
print (yhat4)

# Compare smoothed and original
i = 0
noiseless_y = list()
noiseless_x = list()
pulses = list()
dy = list()
bcs = list()
while i < len(zc_x):
    j = i - 1
    average = 0
    
    # Find closely grouped points and clump them together
    while j < len(zc_x) and numpy.sqrt((zc_x[j] - zc_x[j - 1])**2 + (zc_y[j] - zc_y[j - 1])**2) <= avg_d:
        
        # Variance between smooth graph and original
        average += abs(zc_y[j] - yhat4[j])
        j += 1
        
    # Filter out pulses that are too small or too noisy
    if (j - i > pulse_size):
        
        if 12 / (j - i) <= cutoff:
        
            # Add pulse lines
            pulses.append(zc_x[i])
            bc = list()

            # Build noiseless graph
            while i < j:
                noiseless_y.append(zc_y[i])
                noiseless_x.append(zc_x[i])
                bc.append([zc_x[i], zc_y[i]])
                i += 1

            # Add pulse lines
            pulses.append(zc_x[i])
            bcs.append(bc)
    i += 1
    
# Graph data
trace_noisy = graphs.Scatter(
                            x = zc_x, 
                            y = zc_y,
                            mode = 'markers',
                            name = 'ZC- noisy'
                        )
trace_smooth = graphs.Scatter(
                            x = zc_x, 
                            y = yhat4,
                            name = 'ZC- smoothed'
                        )
trace_noiseless = graphs.Scatter(
                            x = noiseless_x, 
                            y = noiseless_y,
                            mode = 'markers',
                            name = 'ZC- noiseless'
                         )

shapes = list()
for i in pulses:
    shapes.append({'type': 'line',
                   'xref': 'x',
                   'yref': 'y',
                   'x0': i,
                   'y0': 0,
                   'x1': i,
                   'y1': 100000,
                   'line': {
                        'color': 'rgb(139, 0, 0)',
                        'width': 1,
                    },
                  })
layout = graphs.Layout(shapes=shapes)
trace = [trace_noisy, trace_smooth, trace_noiseless]
figure={
    'data': trace,
    'layout': layout
   }
py.iplot(figure, filename=filename)
# return bcs

[[0.00000000e+00 3.84530319e+04]
 [8.90000000e-05 3.84503495e+04]
 [1.74000000e-04 3.84477891e+04]
 ...
 [1.05614250e+01 1.11810486e+04]
 [1.05619430e+01 1.11803307e+04]
 [1.05623480e+01 1.11797694e+04]]


In [68]:
# Approach 6: Hybrid of 1.1 and 3.1
# 1. Chunk graph into pulses
# 2. Get derivative of graph
# 3. Smooth random holes
# 4. Remove points that are too different from their neighbors
# 5. Remove points that are too different from a smoothed interpretation of the graph

import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
from scipy import interpolate
from scipy.signal import savgol_filter

def clean_zc_graph(filename ='P7132033_37', dy_cutoff = 2000, dx_cutoff = .2, pulse_size = 20):
    # Load file into 2d list
    with open('../data/'+ filename + '.csv', 'r') as f:
        reader = csv.reader(f)
        next(reader)
        zc_str = list(reader)

    # Format zc_str to floats
    zc_x = list()
    zc_y = list()
    for x, y in zc_str:
            zc_x.append(float(x) * 1000)
            zc_y.append(float(y))

    # Distance functions
    def dist(ax, ay, bx, by):
        return numpy.sqrt((ax - bx)**2 + (ay - by)**2)

    def dista(pair):
        return dist(pair[0][0], pair[0][1], pair[1][0], pair[1][1])

    # Identify pulses
    graph = list()
    pulse = list()
    prev_x = 0
    for x, y in zip(zc_x, zc_y):
        if x - prev_x <= dx_cutoff:
            pulse.append([x, y])
        elif len(pulse) < pulse_size:
            pulse = [[x, y]]
        else:
            graph.append(pulse)
            pulse = [[x, y]]
        prev_x = x

    # Get 1st derivative
    graph_dy = list()
    prev_y = 0
    for pulse in graph:
        dy = list()
        for x, y in pulse:
            dy.append(abs(y - prev_y))
            prev_y = y
        graph_dy.append(dy)

    # Smooth holes
    for dy, pulse in zip(graph_dy, graph):
        i = 1
        while i < (len(dy) - 2):
            if dy[i] > dy_cutoff:
                if dy[i - 1] < dy_cutoff:
                    if dy[i + 1] < dy_cutoff:
                        pulse[i][1] = (pulse[i - 1][1] + pulse[i + 1][1])/2
                    elif dy[i + 2] < dy_cutoff:
                        pulse[i][1] = (pulse[i - 1][1] + pulse[i + 2][1])/2
                elif dy[i - 2] < dy_cutoff:
                    if dy[i + 1] < dy_cutoff:
                        pulse[i][1] = (pulse[i - 2][1] + pulse[i + 1][1])/2
                    elif dy[i + 2] < dy_cutoff:
                        pulse[i][1] = (pulse[i - 2][1] + pulse[i + 2][1])/2
            i += 1

    # Clean pulses
    clean_graph = list()
    for k, pulse in enumerate(graph):
        i = 1
        while i < len(pulse):
            j = i

            # Count neighboring points
            while j < len(pulse) - 1 and graph_dy[k][j] <= dy_cutoff:
                j += 1

            # If there are enough neighbors, it's good
            if j - i >= pulse_size:
                clean_graph.append(pulse[i:j])

            i = j + 1

    # Clean pulses more
    cleaner_graph = list()
    smooth_graph = list()
    for pulse in clean_graph:

        # Build smooth graph using Savitzky-Golay filter
        # Left param is all x values in current pulse, right param is smoothed y values
        # Params are zipped together then converted to list
        # This is the dark side of pythonic code
        smooth_pulse = list(zip([point[0] for point in pulse], savgol_filter([point[1] for point in pulse], 17, 3)))
        smooth_graph.extend(smooth_pulse)

        # Build clean pulse
        # Iterate through zipped list of smooth_pulse and pulse, producing [[ax, ay],[bx, by]]
        # Keep only those where the absolute distance between pair 1 and pair 2 is less than 1/2 dy_cutoff
        # This is the even darker side of pythonic code
        cleaner_graph.extend([pair[0] for pair in zip(pulse, smooth_pulse) if dista(pair) < dy_cutoff / 2])

    graph_x = list()
    graph_y = list()
    for pulse in graph:
        for point in pulse:
            graph_x.append(point[0])
            graph_y.append(point[1])

    # Graph data
    trace_noisy = graphs.Scatter(
                                x = graph_x, 
                                y = graph_y,
                                mode = 'markers',
                                name = 'ZC- noisy'
                            )
    trace_smooth = graphs.Scatter(
                                x = [point[0] for point in smooth_graph], 
                                y = [point[1] for point in smooth_graph],
                                name = 'ZC- smoothed'
                            )
    trace_noiseless = graphs.Scatter(
                                x = [point[0] for point in cleaner_graph], 
                                y = [point[1] for point in cleaner_graph],
                                mode = 'markers',
                                name = 'ZC- noiseless'
                             )

    trace = [trace_noisy, trace_smooth, trace_noiseless]
    figure={
        'data': trace
       }
    py.iplot(figure, filename=filename)
    return cleaner_graph


[[349.193, 47619.048], [349.279, 46783.626], [349.365, 46511.628], [349.451, 46511.628], [349.537, 46511.628], [349.623, 46511.628], [349.711, 45977.011], [349.798, 45714.286], [349.88399999999996, 46242.775], [349.971, 46242.775], [350.058, 45977.011], [350.145, 45977.011], [350.231, 46242.775], [350.31800000000004, 46242.775], [350.40500000000003, 45977.011], [350.492, 45977.011], [350.58, 45714.286], [350.668, 45454.545], [350.754, 45977.011], [350.841, 46242.775], [350.928, 45977.011], [351.016, 45714.286], [351.10400000000004, 45454.545], [351.191, 45714.286], [351.28, 45454.545], [449.073, 47058.824], [449.158, 47058.824], [449.245, 46511.628], [449.33, 46511.628], [449.415, 47058.824], [449.502, 46511.628], [449.589, 45977.011], [449.675, 46242.775], [449.761, 46511.628], [449.846, 46783.626], [449.932, 46783.626], [450.018, 46511.628], [450.10499999999996, 46242.775], [450.19100000000003, 46242.775], [450.278, 46242.775], [450.365, 45977.011], [450.45099999999996, 46242.775], [

In [4]:
import csv, numpy
import plotly.plotly as py
import plotly.graph_objs as graphs
import numpy as np

# Load file into 2d list
filename = 'P7132033_37'
with open('../data/'+ filename + '.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    zc_str = list(reader)

# Format zc_str to floats and remove low frequency values
zc_x = list()
zc_y = list()
zc = list()
for x, y in zc_str:
        zc_x.append(float(x))
        zc_y.append(float(y))
        zc.append([float(x), float(y)])

dist = list()
j = 1
while j < len(zc_x):
    dist.append(numpy.sqrt((zc_x[j] - zc_x[j - 1])**2 + (zc_y[j] - zc_y[j - 1])**2))
    j+=1
    
neighbors = list()
i = 1
while i < len(zc_x):
    j = i
    count = 0
    while numpy.sqrt((zc_x[j] - zc_x[j - 1])**2 + (zc_y[j] - zc_y[j - 1])**2) <= 1250:
        count += 1
        j += 1
    neighbors.append(count)
    i += 1

data = [graphs.Histogram(x=dist)]
py.iplot(data, filename = "testHisto")


In [ ]:
# Smooth graph- Savitsky-Golay filter
yhat2 = savgol_filter(zc_y, 27, 2)
yhat3 = savgol_filter(zc_y, 17, 3)
yhat4 = savgol_filter(zc_y, 17, 4) # best fit so far

# Interesting idea worth exploring further, not useful at present
def recurs_neighbors(initial, curr, neighbors):
    if curr is initial[-1]:
        return neighbors
    
    temp = list()
    for x, y in initial:
        if dist(curr[0], curr[1], x, y) < density and [x, y] not in neighbors:
            temp.append([x, y])   
    
    if(len(temp) > 2):
        curr = temp[-1]
        neighbors.extend(temp)
        return recurs_neighbors(initial, curr, neighbors)
    print(neighbors)
    return neighbors

In [7]:
import plotly
plotly.tools.set_credentials_file(username='souhad', api_key='L69mhJTzSXbVZRpmZXr3')